In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from math import ceil

data = pd.read_excel('data.xlsx', header=None).values.flatten()

In [2]:
requirements = data
requirements

array([ 11,   5,   4,   7,  16,   6,   5,   7,  13,   6,   5,   7,  12,
         5,   4,   6,   9,   5,   5,  11,  29,  21,  17,  20,  27,  13,
         9,  10,  16,   6,   5,   7,  11,   5,   5,   6,  12,   7,   7,
        10,  15,  10,   9,  11,  15,  10,  10,  16,  26,  21,  23,  36,
        50,  45,  45,  49,  57,  43,  40,  44,  52,  43,  42,  45,  52,
        41,  39,  41,  48,  35,  34,  35,  42,  34,  36,  43,  55,  48,
        54,  65,  80,  70,  74,  85, 101,  89,  88,  90, 100,  87,  88,
        89, 104,  89,  89,  90, 106,  96,  94,  99, 109,  99,  96, 102],
      dtype=int64)

In [3]:
total_week = 64
requirements = requirements[0:total_week]
K = 20
waste_rate = 0.1

In [4]:
working_hands = cp.Parameter(total_week)
waste_hands = cp.Parameter(total_week)
working_hands.value = 4 * requirements
waste_hands.value = np.array([
    round(4 * x * waste_rate) if x % 1 != 0.5 else ceil(4 * x * waste_rate) 
    for x in requirements
])

teaching_hands = cp.Variable(total_week, integer=True)
resting_hands = cp.Variable(total_week, integer=True)
new_hands = cp.Variable((3, total_week), integer=True)

# 机械手购买的分段点
hand_price = cp.Parameter((3, 1))
hand_price.value = np.array([100, 90, 80]).reshape(3, 1)

hand_y = cp.Variable((3, total_week), boolean=True)

In [5]:
working_hands.value, waste_hands.value

(array([ 44,  20,  16,  28,  64,  24,  20,  28,  52,  24,  20,  28,  48,
         20,  16,  24,  36,  20,  20,  44, 116,  84,  68,  80, 108,  52,
         36,  40,  64,  24,  20,  28,  44,  20,  20,  24,  48,  28,  28,
         40,  60,  40,  36,  44,  60,  40,  40,  64, 104,  84,  92, 144,
        200, 180, 180, 196, 228, 172, 160, 176, 208, 172, 168, 180],
       dtype=int64),
 array([ 4,  2,  2,  3,  6,  2,  2,  3,  5,  2,  2,  3,  5,  2,  2,  2,  4,
         2,  2,  4, 12,  8,  7,  8, 11,  5,  4,  4,  6,  2,  2,  3,  4,  2,
         2,  2,  5,  3,  3,  4,  6,  4,  4,  4,  6,  4,  4,  6, 10,  8,  9,
        14, 20, 18, 18, 20, 23, 17, 16, 18, 21, 17, 17, 18]))

In [6]:
cons = [
    resting_hands[0] + teaching_hands[0] + working_hands[0] == 50,
    K * teaching_hands >= cp.sum(new_hands, axis=0),

    teaching_hands >= 0,
    resting_hands >= 0,
    new_hands >= 0,

    # 分段函数引入的约束
    # hand_y >= 0,
    # hand_y <= 1,
    20 * hand_y[1] <= new_hands[0],
    new_hands[0] <= 20 * hand_y[0],
    20 * hand_y[2] <= new_hands[1],
    new_hands[1]  <= 20 * hand_y[1],
    new_hands[2] <= 10000 * hand_y[2]
]

In [7]:
for t in range(total_week - 1):
    cons.append(resting_hands[t + 1] >= working_hands[t] - waste_hands[t]),
    cons.append(resting_hands[t + 1] + working_hands[t + 1] + teaching_hands[t + 1] == \
        resting_hands[t] + working_hands[t] + teaching_hands[t] + cp.sum(new_hands, axis=0)[t] - waste_hands[t]
        )

In [8]:
obj = cp.Minimize(
    cp.sum(cp.multiply(new_hands, hand_price))\
    + 5 * cp.sum(resting_hands)\
    + 10 * cp.sum(teaching_hands)\
    + 10 * cp.sum(new_hands)
)

In [15]:
prob = cp.Problem(obj, cons)
prob.solve(solver=cp.MOSEK, verbose=True)

                                     CVXPY                                     
                                     v1.2.0                                    
(CVXPY) May 03 09:18:21 AM: Your problem has 512 variables, 136 constraints, and 131 parameters.
(CVXPY) May 03 09:18:21 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 03 09:18:21 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 03 09:18:21 AM: Compiling problem (target solver=MOSEK).
(CVXPY) May 03 09:18:21 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> MOSEK
(CVXPY) May 03 09:18:21 AM: Applying reduction Dcp2Cone
(CVXPY) May 03 09:18:21 AM: Applying reduction CvxAttr2Constr
(

100174.99999999978

In [10]:
working_hands.value, np.array(resting_hands.value, dtype=np.int32), waste_hands.value

(array([ 44,  20,  16,  28,  64,  24,  20,  28,  52,  24,  20,  28,  48,
         20,  16,  24,  36,  20,  20,  44, 116,  84,  68,  80, 108,  52,
         36,  40,  64,  24,  20,  28,  44,  20,  20,  24,  48,  28,  28,
         40,  60,  40,  36,  44,  60,  40,  40,  64, 104,  84,  92, 144,
        200, 180, 180, 196, 228, 172, 160, 176, 208, 172, 168, 180],
       dtype=int64),
 array([  5,  40,  42,  26,  25,  59,  61,  50,  25,  48,  50,  39,  25,
         48,  50,  40,  26,  38,  35,  18,  84, 104, 112,  92,  72, 117,
        128, 120,  92, 126, 128, 118,  99, 119, 117, 111,  85, 100,  97,
         82,  58,  72,  72,  60,  40,  54,  49,  36,  86,  96,  76,  83,
        185, 185, 162, 228, 176, 209, 204, 168, 178, 193, 180, 151]),
 array([ 4,  2,  2,  3,  6,  2,  2,  3,  5,  2,  2,  3,  5,  2,  2,  2,  4,
         2,  2,  4, 12,  8,  7,  8, 11,  5,  4,  4,  6,  2,  2,  3,  4,  2,
         2,  2,  5,  3,  3,  4,  6,  4,  4,  4,  6,  4,  4,  6, 10,  8,  9,
        14, 20, 18, 18, 20, 

In [11]:
np.array(hand_y.value, dtype=np.int32)

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]])

In [12]:
np.array(new_hands.value, dtype=np.int32), np.array(teaching_hands.value, dtype=np.int32)

(array([[ 14,   0,   0,  20,   0,   0,   0,   0,   0,   0,   0,   7,   0,
           0,   0,   0,   0,   0,  15,  20,   0,   0,   0,  15,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,  19,  20,   0,   0,  20,  20,
           0,   0,  20,   0,   0,   0,   0,  20,   0,   0,   0,   0],
        [  0,   0,   0,  16,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,  20,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,  20,   0,   0,  20,  20,
           0,   0,  20,   0,   0,   0,   0,  20,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,  95,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   